In [1]:
import pandas as pd
import numpy as np


In [2]:
#item = pd.read_csv('/home/piai/실습파일/data/on_items.csv', encoding='cp949')

In [3]:
#item.head()

In [4]:
#item['ItemMiddleName'].value_counts()

## 배송지연예측 모델링

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# 파일 경로 지정
file_path = '/content/df_clean (2).csv'

# 파일 읽기 (날짜형 컬럼 자동 파싱 시도)
df = pd.read_csv(file_path, parse_dates=['OrderDT', 'DeliveryDT'])

# 데이터프레임 크기 및 샘플 확인
print(df.shape)
print(df.head(3))


(109089, 32)
   idUser Gender  Age AgeGroup  FamilyCount MemberYN            idOrder  \
0  U10001     여성   26      20대            2        Y  U10001-O2022-1002   
1  U10001     여성   26      20대            2        Y  U10001-O2022-1002   
2  U10001     여성   26      20대            2        Y  U10001-O2022-1002   

              OrderDT  OrderMonth  OrderDay  ...          DeliveryDT  \
0 2022-01-06 17:08:00         1.0       6.0  ... 2022-01-07 06:24:00   
1 2022-01-06 17:08:00         1.0       6.0  ... 2022-01-07 06:24:00   
2 2022-01-06 17:08:00         1.0       6.0  ... 2022-01-07 06:24:00   

   Deli_Year  Deli_Month Deli_Day Deli_Hour Deli_Minute Deli_Second  \
0     2022.0         1.0      7.0       6.0        24.0         0.0   
1     2022.0         1.0      7.0       6.0        24.0         0.0   
2     2022.0         1.0      7.0       6.0        24.0         0.0   

     Deli_Duration  Deli_Hours  IsMorningDelivery  
0  0 days 13:16:00   13.266667                1.0  
1  0 day

In [6]:
df.columns

Index(['idUser', 'Gender', 'Age', 'AgeGroup', 'FamilyCount', 'MemberYN',
       'idOrder', 'OrderDT', 'OrderMonth', 'OrderDay', 'OrderHour',
       'OrderMinute', 'OrderSecond', 'ItemCode', 'ItemLargeName',
       'ItemMiddleName', 'ItemSmallName', 'ItemName', 'Price', 'PriceYear',
       'PriceMin', 'PriceMax', 'DeliveryDT', 'Deli_Year', 'Deli_Month',
       'Deli_Day', 'Deli_Hour', 'Deli_Minute', 'Deli_Second', 'Deli_Duration',
       'Deli_Hours', 'IsMorningDelivery'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109089 entries, 0 to 109088
Data columns (total 32 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   idUser             109089 non-null  object        
 1   Gender             109089 non-null  object        
 2   Age                109089 non-null  int64         
 3   AgeGroup           109089 non-null  object        
 4   FamilyCount        109089 non-null  int64         
 5   MemberYN           109089 non-null  object        
 6   idOrder            109089 non-null  object        
 7   OrderDT            109088 non-null  datetime64[ns]
 8   OrderMonth         109088 non-null  float64       
 9   OrderDay           109088 non-null  float64       
 10  OrderHour          109088 non-null  float64       
 11  OrderMinute        109088 non-null  float64       
 12  OrderSecond        109088 non-null  float64       
 13  ItemCode           109088 non-null  object  

In [8]:
df.drop(['IsMorningDelivery',
         'OrderMonth', 'OrderDay', 'OrderHour', 'OrderMinute', 'OrderSecond',
         'Deli_Year', 'Deli_Month','Deli_Day', 'Deli_Hour', 'Deli_Minute', 'Deli_Second', 'Deli_Duration',
       'Deli_Hours'], axis=1, inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109089 entries, 0 to 109088
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   idUser          109089 non-null  object        
 1   Gender          109089 non-null  object        
 2   Age             109089 non-null  int64         
 3   AgeGroup        109089 non-null  object        
 4   FamilyCount     109089 non-null  int64         
 5   MemberYN        109089 non-null  object        
 6   idOrder         109089 non-null  object        
 7   OrderDT         109088 non-null  datetime64[ns]
 8   ItemCode        109088 non-null  object        
 9   ItemLargeName   109088 non-null  object        
 10  ItemMiddleName  109088 non-null  object        
 11  ItemSmallName   109088 non-null  object        
 12  ItemName        109088 non-null  object        
 13  Price           109088 non-null  float64       
 14  PriceYear       109088 non-null  flo

In [10]:
# 1. 배송 지연 여부 판단 함수
from datetime import timedelta

def check_delivery(order, delivery):
    order_hour = order.hour
    order_weekday = order.weekday()  # 월=0, ..., 일=6

    if order_weekday == 4 and order_hour >= 21:  # 금요일 21시 이후
        deadline = order.replace(hour=7, minute=0, second=0) + timedelta(days=3)
    elif order_weekday == 6 and order_hour >= 21:  # 일요일 21시 이후
        deadline = order.replace(hour=7, minute=0, second=0) + timedelta(days=2)
    elif (order_weekday == 5) or (order_weekday == 6 and order_hour < 21):
        if order_weekday == 5:  # 토요일
            deadline = order.replace(hour=7, minute=0, second=0) + timedelta(days=2)
        else:  # 일요일 21시 이전
            deadline = order.replace(hour=7, minute=0, second=0) + timedelta(days=1)
    else:
        if order_hour < 21:
            deadline = order.replace(hour=7, minute=0, second=0) + timedelta(days=1)
        else:
            deadline = order.replace(hour=7, minute=0, second=0) + timedelta(days=2)

    return 0 if delivery <= deadline else 1

# 2. 냉장/냉동/상온 파생 변수용 중분류 매핑
def map_storage_type(middle_name):
    냉동 = ['냉동', '수산']
    냉장 = ['신선', '축산', '과일', '채소']
    상온 = ['곡물', '과자', '음료', '탄산', '생수', '커피', '즉석', '양념']

    if pd.isnull(middle_name): return '기타'
    for 키, 값 in {'냉동': 냉동, '냉장': 냉장, '상온': 상온}.items():
        if any(카테고리 in middle_name for 카테고리 in 값):
            return 키
    return '기타'

# 3. 파생 피처 생성
df['OrderHour'] = df['OrderDT'].dt.hour
df['OrderWeekday'] = df['OrderDT'].dt.weekday
df['DeliveryHour'] = df['DeliveryDT'].dt.hour
df['DeliveryWeekday'] = df['DeliveryDT'].dt.weekday
df['OrderToDeliveryMinutes'] = (df['DeliveryDT'] - df['OrderDT']).dt.total_seconds() / 60

df['IsWeekendOrder'] = df['OrderWeekday'].isin([5, 6]).astype(int)
df['PeakHour'] = df['OrderHour'].between(18, 21).astype(int)
df['StorageType'] = df['ItemMiddleName'].apply(map_storage_type)

# 4. 라벨 생성
df['LateYN'] = df.apply(lambda row: check_delivery(row['OrderDT'], row['DeliveryDT']), axis=1)

# 5. 주문량 구간 (정규화 또는 quantile 방식 가능)
order_volume_q = df.groupby(df['OrderDT'].dt.date)['idOrder'].nunique()
order_volume_q = pd.qcut(order_volume_q, q=3, labels=['Low', 'Mid', 'High']).to_dict()
df['OrderDate'] = df['OrderDT'].dt.date
df['OrderVolumeLevel'] = df['OrderDate'].map(order_volume_q)

# 결과 확인
print(df[['OrderDT', 'DeliveryDT', 'LateYN', 'StorageType', 'OrderVolumeLevel']].head())


              OrderDT          DeliveryDT  LateYN StorageType OrderVolumeLevel
0 2022-01-06 17:08:00 2022-01-07 06:24:00       0          냉동             High
1 2022-01-06 17:08:00 2022-01-07 06:24:00       0          상온             High
2 2022-01-06 17:08:00 2022-01-07 06:24:00       0          냉동             High
3 2022-01-06 17:08:00 2022-01-07 06:24:00       0          냉장             High
4 2022-01-13 16:50:00 2022-01-14 06:28:00       0          냉장             High


In [11]:
df.columns

Index(['idUser', 'Gender', 'Age', 'AgeGroup', 'FamilyCount', 'MemberYN',
       'idOrder', 'OrderDT', 'ItemCode', 'ItemLargeName', 'ItemMiddleName',
       'ItemSmallName', 'ItemName', 'Price', 'PriceYear', 'PriceMin',
       'PriceMax', 'DeliveryDT', 'OrderHour', 'OrderWeekday', 'DeliveryHour',
       'DeliveryWeekday', 'OrderToDeliveryMinutes', 'IsWeekendOrder',
       'PeakHour', 'StorageType', 'LateYN', 'OrderDate', 'OrderVolumeLevel'],
      dtype='object')

In [12]:
# 주문 단위 요약 함수
def summarize_order_level(df):
    order_df = df.groupby('idOrder').agg({
        'OrderDT': 'first',
        'DeliveryDT': 'first',
        'Price': 'mean',
        'OrderHour': 'first',
        'OrderWeekday': 'first',
        'IsWeekendOrder': 'first',
        'PeakHour': 'first',
        'StorageType': lambda x: x.mode()[0] if not x.mode().empty else '기타',
        'OrderVolumeLevel': 'first',
        'LateYN': 'max'  # 하나라도 지연이면 전체 지연
    }).reset_index()

    return order_df

# 1. 주문 단위 요약
df_order = summarize_order_level(df)

# 2. 인코딩: StorageType → One-hot, OrderVolumeLevel → Ordinal
df_order = pd.get_dummies(df_order, columns=['StorageType'], drop_first=True)

volume_map = {'Low': 0, 'Mid': 1, 'High': 2}
df_order['OrderVolumeLevel'] = df_order['OrderVolumeLevel'].map(volume_map)


df_order.drop(['idOrder','Price', 'OrderHour', 'DeliveryDT', 'OrderDT',
              ], axis=1, inplace=True)

df_order["OrderToDeliveryMinutes"] = df['OrderToDeliveryMinutes']
print(df_order.info())

# 3. 피처 / 타깃 분리
X = df_order.drop(columns=['LateYN'])  # id와 시간은 제외
y = df_order['LateYN']

# 4. 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("✅ 주문 단위로 분리 완료!")
#print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21874 entries, 0 to 21873
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   OrderWeekday            21873 non-null  float64
 1   IsWeekendOrder          21874 non-null  int64  
 2   PeakHour                21874 non-null  int64  
 3   OrderVolumeLevel        21873 non-null  float64
 4   LateYN                  21874 non-null  int64  
 5   StorageType_냉동          21874 non-null  bool   
 6   StorageType_냉장          21874 non-null  bool   
 7   StorageType_상온          21874 non-null  bool   
 8   OrderToDeliveryMinutes  21874 non-null  float64
dtypes: bool(3), float64(3), int64(3)
memory usage: 1.1 MB
None
✅ 주문 단위로 분리 완료!


In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15311 entries, 21596 to 15795
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   OrderWeekday            15310 non-null  float64
 1   IsWeekendOrder          15311 non-null  int64  
 2   PeakHour                15311 non-null  int64  
 3   OrderVolumeLevel        15310 non-null  float64
 4   StorageType_냉동          15311 non-null  bool   
 5   StorageType_냉장          15311 non-null  bool   
 6   StorageType_상온          15311 non-null  bool   
 7   OrderToDeliveryMinutes  15311 non-null  float64
dtypes: bool(3), float64(3), int64(2)
memory usage: 762.6 KB


In [14]:
# drop_cols = [
#     'idUser', 'Age', 'idOrder', 'OrderDT', 'ItemCode', 'ItemLargeName',
#     'ItemMiddleName', 'ItemSmallName', 'ItemName', 'PriceYear', 'FamilyCount', 'Gender', 'MemberYN',
#     'PriceMin', 'PriceMax', 'DeliveryDT', 'DeliveryHour', 'DeliveryWeekday',
#     'OrderToDeliveryMinutes', 'OrderDate'
# ]

# df_model = df.drop(columns=drop_cols)




In [15]:
# df_model.info()

### 데이터셋 분리

In [16]:
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd

def under_sampling(df, target_col):
    """
    언더샘플링을 통해 데이터 불균형을 해결하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        target_col (str): 목표 변수 컬럼명 (0과 1로 구성된 이진 분류)

    Returns:
        pd.DataFrame: 언더샘플링된 데이터프레임
    """
    # 클래스별 데이터 나누기
    df_majority = df[df[target_col] == 0]
    df_minority = df[df[target_col] == 1]

    # 소수 클래스의 샘플 수
    minority_count = len(df_minority)

    # 다수 클래스를 소수 클래스 수에 맞춰 랜덤 샘플링
    df_majority_downsampled = resample(df_majority,
                                       replace=False,     # 복원 샘플링하지 않음
                                       n_samples=minority_count, # 소수 클래스 수만큼 샘플링
                                       random_state=42)   # 재현성 확보

    # 소수 클래스와 다수 클래스 데이터를 결합
    df_balanced = pd.concat([df_majority_downsampled, df_minority])

    # 데이터 셔플링
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    return df_balanced

balanced_df = under_sampling(df_order, "LateYN")

# 결과 확인
print("언더샘플링 전 데이터 분포:")
print(df_order["LateYN"].value_counts())

print("\n언더샘플링 후 데이터 분포:")
print(balanced_df["LateYN"].value_counts())

print(balanced_df.head())

X = df_order.drop(columns=['LateYN'])  # id와 시간은 제외
y = df_order['LateYN']

# 5. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 학습
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("정확도:", accuracy_score(y_test, y_pred))
print("분류 보고서:\n", classification_report(y_test, y_pred))


언더샘플링 전 데이터 분포:
LateYN
0    20955
1      919
Name: count, dtype: int64

언더샘플링 후 데이터 분포:
LateYN
1    919
0    919
Name: count, dtype: int64
   OrderWeekday  IsWeekendOrder  PeakHour  OrderVolumeLevel  LateYN  \
0           5.0               1         0               0.0       1   
1           6.0               1         0               2.0       1   
2           3.0               0         1               2.0       0   
3           3.0               0         1               0.0       0   
4           0.0               0         0               1.0       1   

   StorageType_냉동  StorageType_냉장  StorageType_상온  OrderToDeliveryMinutes  
0           False            True           False                   820.0  
1           False            True           False                  1201.0  
2           False            True           False                   546.0  
3           False            True           False                   625.0  
4           False            True           False     

In [17]:

X = df_order.drop(columns=['LateYN'])  # id와 시간은 제외
y = df_order['LateYN']
print(X.columns)
print(X.describe)
# 5. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 모델 학습
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("정확도:", accuracy_score(y_test, y_pred))
print("분류 보고서:\n", classification_report(y_test, y_pred))


Index(['OrderWeekday', 'IsWeekendOrder', 'PeakHour', 'OrderVolumeLevel',
       'StorageType_냉동', 'StorageType_냉장', 'StorageType_상온',
       'OrderToDeliveryMinutes'],
      dtype='object')
<bound method NDFrame.describe of        OrderWeekday  IsWeekendOrder  PeakHour  OrderVolumeLevel  \
0               4.0               0         1               0.0   
1               3.0               0         0               2.0   
2               3.0               0         0               2.0   
3               3.0               0         1               2.0   
4               3.0               0         1               2.0   
...             ...             ...       ...               ...   
21869           0.0               0         1               2.0   
21870           0.0               0         1               2.0   
21871           0.0               0         0               2.0   
21872           6.0               1         0               2.0   
21873           6.0               1    

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(2048, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(2048, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류를 위해 sigmoid 사용
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # 확률을 0 또는 1로 변환

print("정확도:", accuracy_score(y_test, y_pred))
print("분류 보고서:\n", classification_report(y_test, y_pred))

Epoch 1/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9589 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 2/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9597 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 3/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9614 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 4/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9609 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 5/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9559 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 6/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9584 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 7/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9573 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 8/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9579 - loss: nan - val_accuracy: 0.9511 - val_loss: nan
Epoch 9/20
438/438 ━━━━━

# 베이스모델_랜덤포레스트

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# 1. 모델 선언 및 학습
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# 2. 검증셋 예측
y_pred = rf_clf.predict(X_test)
#y_proba = rf_clf.predict_proba(X_val)[:, 1]  # ROC-AUC용 확률

# 3. 평가 결과 출력
print("📊 Classification Report (Validation Set)")
print(classification_report(y_test, y_pred))

print("📉 Confusion Matrix")
print(confusion_matrix(y_test, y_pred))

print(f"🎯 ROC-AUC: {roc_auc_score(y_test, y_pred):.4f}")


📊 Classification Report (Validation Set)
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4195
           1       0.04      0.02      0.03       180

    accuracy                           0.94      4375
   macro avg       0.50      0.50      0.50      4375
weighted avg       0.92      0.94      0.93      4375

📉 Confusion Matrix
[[4101   94]
 [ 176    4]]
🎯 ROC-AUC: 0.4999


## 오버샘플링

In [20]:
from imblearn.over_sampling import SMOTE

# 오버샘플링
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("🔄 After SMOTE oversampling:")
print(pd.Series(y_train_smote).value_counts())


ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
rf_smote.fit(X_train_smote, y_train_smote)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

y_pred = rf_smote.predict(X_val)
y_proba = rf_smote.predict_proba(X_val)[:, 1]

print("📊 Classification Report (Validation Set - SMOTE)")
print(classification_report(y_val, y_pred))

print("📉 Confusion Matrix")
print(confusion_matrix(y_val, y_pred))

print(f"🎯 ROC-AUC: {roc_auc_score(y_val, y_proba):.4f}")


## LightGBM

In [ ]:
pip install lightgbm

In [ ]:
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# LightGBM용 데이터 준비
lgbm_clf = lgb.LGBMClassifier(random_state=42)

# 학습 (SMOTE 데이터 기준)
lgbm_clf.fit(X_train_smote, y_train_smote)

# 예측
y_pred = lgbm_clf.predict(X_val)
y_proba = lgbm_clf.predict_proba(X_val)[:, 1]

# 평가
print("📊 Classification Report (LightGBM)")
print(classification_report(y_val, y_pred))
print("📉 Confusion Matrix")
print(confusion_matrix(y_val, y_pred))
print(f"🎯 ROC-AUC: {roc_auc_score(y_val, y_proba):.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

feature_imp = pd.Series(lgbm_clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=feature_imp[:15], y=feature_imp[:15].index)
plt.title('💡 Feature Importance (LightGBM)')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


## XGBoost

In [ ]:
pip install xgboost


In [ ]:
from os import XATTR_REPLACE
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# DMatrix로 변환 (optional but 권장)
dtrain = xgb.DMatrix(X_train_smote, label=y_train_smote)
dval = xgb.DMatrix(XATTR_REPLACE)

# 모델 파라미터 기본값
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42,
    'scale_pos_weight': 24  # 클래스 불균형 보정용 (정시:지연 비율)
}

# 모델 학습
bst = xgb.train(params, dtrain, num_boost_round=100)

# 예측
y_proba = bst.predict(dval)
y_pred = (y_proba >= 0.5).astype(int)


In [ ]:
print("📊 Classification Report (XGBoost)")
print(classification_report(y_val, y_pred))

print("📉 Confusion Matrix")
print(confusion_matrix(y_val, y_pred))

print(f"🎯 ROC-AUC: {roc_auc_score(y_val, y_proba):.4f}")


In [ ]:
xgb.plot_importance(bst, max_num_features=15, importance_type='gain')
plt.title("📌 Feature Importance (XGBoost)")
plt.show()


## LightGBM 튜닝

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

# LightGBM 모델
lgbm = lgb.LGBMClassifier(random_state=42)

# 파라미터 그리드
param_dist = {
    'num_leaves': [16, 32, 64, 128],
    'max_depth': [3, 5, 7, 10],
    'min_child_samples': [10, 20, 30, 50],
    'learning_rate': [0.01, 0.05, 0.1],
    'scale_pos_weight': [10, 15, 20, 25]
}

# 튜닝
rsearch = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=param_dist,
    n_iter=30,  # 실행 횟수 (늘릴수록 정확, 오래 걸림)
    scoring='f1',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# SMOTE 데이터 기준 학습
rsearch.fit(X_train_smote, y_train_smote)

# 결과
print("✅ Best Params:", rsearch.best_params_)
print("🎯 Best Score (F1):", rsearch.best_score_)


In [ ]:
best_lgbm = rsearch.best_estimator_

# 예측
y_pred = best_lgbm.predict(X_val)
y_proba = best_lgbm.predict_proba(X_val)[:, 1]

# 평가
print("📊 Classification Report (Tuned LightGBM)")
print(classification_report(y_val, y_pred))
print("📉 Confusion Matrix")
print(confusion_matrix(y_val, y_pred))
print(f"🎯 ROC-AUC: {roc_auc_score(y_val, y_proba):.4f}")


지연예측 성능은 향상, 정시배송을 잘못 예측함

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# 예측 확률 기반 Threshold 조정
thresholds = [0.3, 0.5, 0.6, 0.7, 0.8]

for t in thresholds:
    print(f"\n🎯 Threshold = {t}")
    y_pred_thresh = (y_proba >= t).astype(int)
    print(classification_report(y_val, y_pred_thresh, zero_division=0))
    print(confusion_matrix(y_val, y_pred_thresh))


In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, threshold_vals = precision_recall_curve(y_val, y_proba)

plt.figure(figsize=(10,5))
plt.plot(threshold_vals, precision[:-1], label='Precision')
plt.plot(threshold_vals, recall[:-1], label='Recall')
plt.axvline(0.5, color='gray', linestyle='--', label='Default=0.5')
plt.title("📈 Precision-Recall vs Threshold")
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.grid()
plt.show()


배송 지연을 설명해줄 피처가 거의 없음
현재 피처:

- 시간 정보 (주문 시각, 요일, 피크시간)

- 상품군 정보 (냉동/냉장/상온)

- 전체 주문 수량

물류 상황, 지역 정보, 라이더 수, 날씨, 거리 정보 없음 → 핵심 변수들 부재

## 배송지연 예측에 필요한 핵심 정보 부족으로 모델링 한계

🧠 주요 분석 및 모델링 흐름
지연 여부 라벨링 (SLA 기준 기반)

주문단위 요약 (idOrder 기준 Aggregation)

파생변수 생성: 시간대, 요일, 피크타임, StorageType, 주문몰림 정도

SMOTE 오버샘플링을 통한 클래스 불균형 보정

RandomForest / LightGBM / XGBoost 모델 비교

Threshold 조정 통한 정밀도-재현율 트레이드오프 탐색

📉 결과 요약
모델	Recall(지연)	Precision(지연)	Accuracy	ROC-AUC	비고

RF + SMOTE	0.15	0.04	0.82	0.51	가장 안정적인 베이스라인

LightGBM + SMOTE	0.19	0.04	0.79	0.49	지연 탐지 ↑, 정시 오차 ↑

XGBoost + SMOTE	0.77	0.04	0.25	0.49	과도한 지연 예측

Tuned LGBM	0.75	0.04	0.25	0.49	비슷한 경향, Precision ↓

📌 결론: 지연 class의 예측 정확도는 높지만 정시 예측력이 크게 훼손, ROC-AUC도 랜덤 수준 → 예측 모델 실효성 부족

🚚 2. 배송 데이터 강화 전략 제안

✅ A. 추가 데이터 수집 아이디어
항목	설명
고객 위치 정보	시/군/구, 도착지까지 거리 등 포함
날씨 정보	지역별 강수, 기온, 한파/폭우 상황
물류센터 처리량	당일 센터별 처리건수, 투입인원 수
라이더 수 / 배정률	배송 인력 대비 주문 수 비율
상품 배송 우선도	냉동/대용량 상품 중심 여부
주문 생성 시점 vs 처리 시점	시스템 내 처리 지연 여부 추적 가능

✅ B. 운영 정책/모델 개선 아이디어
전략	설명
지역별 배송지연률 시각화	물류망 개선 기준 제시 가능
고위험 조합 조건 룰 기반 알림	“금요일 20시+냉동+5개 이상” 주문 위험도 표시
예측 모델 → 위험 점수화	“지연 예측 확률이 높은 주문”에 가중 처리 적용
데이터 로그 강화 정책 도입	물류센터, 라이더, 기상, 지역정보 자동 기록 시스템 구축

📌 최종 제안
예측 모델은 "지금 당장 운영에 활용하기엔 부족"하지만,
✨ 분석을 통해 데이터 구조, 필요한 피처, 모델의 한계를 명확히 파악했고
✨ 이제는 데이터 수집 → 분석 자동화 → 예측모델 고도화 단계로 넘어갈 준비가 되었다!